# Scraping Amazon website for price drop and sending an email...

### Importing libraries

In [1]:
# importing Library

from bs4 import BeautifulSoup
import requests
import time
import datetime

### Conncting to Amazon website

In [2]:
# connect to wesite

URL = "https://www.amazon.in/OnePlus-Eternal-Storage-Amazon-Credit/dp/B0BQJM2PXW/ref=sr_1_5?qid=1677430587&refinements=p_89%3AOnePlus&rnid=3837712031&s=electronics&sr=1-5&th=1"

headers = ({ "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.56"})

# Making the HTTP Request
page = requests.get(URL, headers = headers)

# If response message 200 is displayed connection is successful
page

<Response [200]>

### Creating "bs4.BeautifulSoup" for scraping our required tags from the wwebsite

In [3]:
# Creating the Soup Object containing all data

soup1 = BeautifulSoup(page.content, "lxml")
soup = BeautifulSoup(soup1.prettify(), "html.parser")

In [4]:
type(soup)

bs4.BeautifulSoup

### Scraping Title

In [5]:
title = soup.find(id = 'productTitle').get_text().strip()
title

'OnePlus 11 5G (Eternal Green, 16GB RAM, 256GB Storage) Upto 18M No Cost EMI with ICICI Amazon Pay Credit Card'

### Scraping Price

In [6]:
# price.strip()[1:] # removes 1st element
price = soup.find("span", attrs = {"class" : 'a-offscreen'}).get_text().strip()[1:].replace(',', '') 
price = float(price)
price

61999.0

### Using timestamp to check extract data time

In [7]:
import datetime

today = datetime.date.today()
print(today)

2023-03-01


### Importing the data into csv format

In [8]:
# importing csv library
import csv 

# declaring header values for our file
header = ['Title', 'Price', 'Date']

# creating a list as "data" having values title, price, and today
data = [title,price, today]

# Creating a new file for storing the scraped data using csv 
with open ('AmazonScrapedData.csv' , 'w', newline = '', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerow(data)

### Using pandas to view the csv

In [10]:
import pandas as pd

df = pd.read_csv(r'C:/Users/Sweekar/AlexAnalyst/AmazonScrapedData.csv')
df

,Title,Price,Date
0,"OnePlus 11 5G (Eternal Green, 16GB RAM, 256GB ...",61999.0,2023-03-01


### Appending the data into csv

In [12]:
# appending the same list "data" values into AmazonScrapedData file using "a+"
# Now, every time we run this a new row is added
with open ('AmazonScrapedData.csv', 'a+', newline = '', encoding='UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(data)

# re reading the file to check updated values
df = pd.read_csv(r'C:/Users/Sweekar/AlexAnalyst/AmazonScrapedData.csv')
df

,Title,Price,Date
0,"OnePlus 11 5G (Eternal Green, 16GB RAM, 256GB ...",61999.0,2023-03-01
1,"OnePlus 11 5G (Eternal Green, 16GB RAM, 256GB ...",61999.0,2023-03-01


### Sending the mail if price drops

In [14]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage

# Sender's email address and password
sender_email = "wellwishermani7262@gmail.com"

# Accessing password from another file for security porposes
file1 = open("wellwisher_gmailPass.txt","r")
data = file1.read()
data_into_list = data.replace('\n', ' ').split(".")
file1.close()

# The password is stored as a list we convert it into string using list comprehension
StringPass = ' '.join([str(elem) for elem in data_into_list])
sender_password = StringPass

# Recipient email address
recipient_email = "sweekar.786b@gmail.com"

# Create a MIMEMultipart object
message = MIMEMultipart()

# Set the email subject, body, and sender/recipients
message['Subject'] = "Price Drop Alert!"
message['From'] = sender_email
message['To'] = recipient_email

# Add a plain text message to the MIMEMultipart object
text = "Hurray! The product you have been looking for is now available at your convenient price: Url: https://www.amazon.in/OnePlus-Eternal-Storage-Amazon-Credit/dp/B0BQJM2PXW/ref=sr_1_5?qid=1677430587&refinements=p_89%3AOnePlus&rnid=3837712031&s=electronics&sr=1-5&th=1"
message.attach(MIMEText(text))

# Add an image attachment to the MIMEMultipart object
# with open("image.jpg", "rb") as f:
#     image = MIMEImage(f.read())
#     message.attach(image)

# Create an SMTP connection object and login with the sender's email and password
smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.starttls()
smtp.login(sender_email, sender_password)

# Send the email
smtp.sendmail(sender_email, recipient_email, message.as_string())

# Close the SMTP connection
smtp.quit()


(221,
 b'2.0.0 closing connection c13-20020a62e80d000000b005a7f8a326a3sm6443281pfi.50 - gsmtp')